In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys

sys.path.insert(0, "..")

import foraging_toolkit as ft

In [69]:
cp = pd.read_pickle('central_park_data/elm_central_park_birds.pkl')

cp['time'] = cp['time'] + 1
cp['bird'] = cp['bird'] + 1
cp.head()


,x,y,time,bird
0,124.700294,72.926094,1,1
1,156.153030,103.940567,1,2
2,76.500145,152.607285,1,3
3,124.667969,72.846458,2,1
4,148.427032,104.434341,2,2


In [70]:
print(cp.shape)
cp["type"] = "central_park_birds1"

def rescale_to_grid(column, size):
    min_value = min(column)
    max_value = max(column)

    normalized = (column - min_value) / (max_value - min_value)
    scaled = np.round((normalized * (size-1))+1).astype(int)
    
    return scaled


#Q what's a sensible grid size?
grid_size = 40

cp["x"] = rescale_to_grid(cp["x"], grid_size)
cp["y"] = rescale_to_grid(cp["y"], grid_size)
cp.head()

(5436, 4)


,x,y,time,bird,type
0,17,12,1,1,central_park_birds1
1,21,17,1,2,central_park_birds1
2,10,26,1,3,central_park_birds1
3,17,12,2,1,central_park_birds1
4,20,17,2,2,central_park_birds1


In [71]:
cp_subset = cp[cp['time'] < 100]

In [72]:
time = list(range(1, cp_subset["time"].max() + 1))

rewards_x = 1
rewards_y = 1


if  isinstance(rewards_x,int):
    time = [t for t in time]
    data = {
        "x": [rewards_x] * len(time),
        "y": [rewards_y] * len(time),
        "time": time
    }
else:
    time = [t for t in time for _ in range(len(rewards_x))],

    data = {
        "x": rewards_x * len(time),
        "y": rewards_y * len(time),
        "time": time
    }

rewardsDF = pd.DataFrame(data)
rewardsDF
#rewardsDF["x"] = rescale_to_grid(rewardsDF["x"], grid_size)
#rewardsDF["y"] = rescale_to_grid(rewardsDF["y"], grid_size)
len(cp_subset["time"].unique())

97

In [73]:
def object_from_data(birdsDF, rewardsDF, grid_size, frames):
    class EmptyObject:
        pass

    sim = EmptyObject()

    sim.grid_size = grid_size
    sim.num_frames = frames
    sim.birdsDF = birdsDF
    sim.rewardsDF = rewardsDF
    sim.birds = [group for _, group in birdsDF.groupby("bird")]
    sim.rewards = [group for _, group in rewardsDF.groupby("time")]
    sim.num_birds = len(sim.birds)

    step_maxes = []

    for b in range(len(sim.birds)):
       
       step_maxes.append (
           max(
               max(
               [abs(sim.birds[b]["x"].iloc[t+1] - sim.birds[b]["x"].iloc[t])  for t in range(0,sim.birds[b].shape[0] - 1)] 
            ),
            max(
               [abs(sim.birds[b]["y"].iloc[t+1] - sim.birds[b]["y"].iloc[t])  for t in range(0,sim.birds[b].shape[0] - 1)] 
            )
           )
        )
    sim.step_size_max = max(step_maxes)   
       
    return sim

cp_obj = object_from_data(cp_subset, rewardsDF, grid_size, 99)


In [85]:
cp_obj.birds[4].head()

,x,y,time,bird,type
85,17,7,24,5,central_park_birds1
89,17,7,25,5,central_park_birds1
93,17,7,26,5,central_park_birds1
97,17,7,27,5,central_park_birds1
101,17,7,28,5,central_park_birds1


In [78]:
ft.animate_birds(cp_obj, plot_rewards=True, width=600, height=600, point_size=10)